In [ ]:
chars = set()
with open("names.txt", "r") as f:
    names = f.read().splitlines()
for name in names:
    chars.update(list(name))
chars = sorted(chars)
chars = ['.'] + list(chars) 
print(chars)


In [ ]:
stoi = {}
idx = 0
for i in range(len(chars)):
    stoi[chars[i]] = idx 
    idx+=1
for char_1 in chars:
    for char_2 in chars:
        stoi[(char_1, char_2)] = idx
        idx  +=1

itos = {i:c for c,i in stoi.items()}
print(stoi)

print(itos)


In [ ]:
import torch 
trigram_tensor = torch.zeros((812,28), dtype=torch.int32)

for name in names:
    chrs = ['.', '.'] + list(name) + ['.']

    for i in range(len(chrs) - 2):
        trigram_tensor[stoi[(chrs[i], chrs[i+1])], stoi[chrs[i+2]]] += 1
  

In [ ]:
trigram_tensor = trigram_tensor.float()
trigram_tensor /=  trigram_tensor.sum(dim = 1, keepdim =  True)




In [ ]:

def sample_from_trigram(trigram_tensor):
    """
    implementation of sampling a value from cumulative probability distribution
    """
    cdf_tensor = trigram_tensor.cumsum(0)
    rand_value = torch.rand(1)
    idx = 0
    for i in range(cdf_tensor.shape[0]):
        if cdf_tensor[i] > rand_value:
            idx = i
            break

    return idx

for i in range(200):
    idx = stoi[('.', '.')]
    word = ['.', '.']
    while True:
        next_idx = sample_from_trigram(trigram_tensor[idx, :])
        next_char = itos[next_idx]

        word.append(next_char)

        idx = stoi[(word[-2], word[-1])]
        if next_idx == 0:
            break
    print("".join(word[2:-1]))

In [ ]:
chars = set()
with open("names.txt", "r") as f:
    names = f.read().splitlines()
for name in names:
    chars.update(list(name))
chars = sorted(chars)
chars = ['.'] + list(chars) 
stoi = {}
idx = 0
for i in range(len(chars)):
    stoi[chars[i]] = idx 
    idx+=1

itos = {i:c for c,i in stoi.items()}
print(stoi)

print(itos)

In [ ]:
import torch.nn.functional as F
import torch
x_feats = []
y_feats = []


for name in names:
    chrs = ['.'] + list(name) + ['.']
    for i in range(len(chrs) - 1):
        x_idx = torch.tensor(stoi[chrs[i]])
        y_idx = torch.tensor(stoi[chrs[i+1]])

        x_feats.append(x_idx)
        y_feats.append(y_idx)
        # print(x_feat, y_feat)
        # print(chrs[i], chrs[i+1])
x_feats = torch.tensor(x_feats)
y_feats = torch.tensor(y_feats)

x_feats = F.one_hot(x_feats, num_classes=len(chars))

x_feats = x_feats.float()

print(x_feats.shape)
print(y_feats.shape)

In [ ]:
# x_feats = (228146, 27)
# W = (27, 27)
# X @ W = (228146, 27) @ (27, 27) = (228146, 27)
W =  torch.randn((27, 27), requires_grad=True)
O = x_feats @ W
print(O.shape)

In [ ]:
print(O[0,:])

In [ ]:

print(O[0,:])
O = O.exp()
O_sum = O.sum(1, keepdim=True)
O_softmax = O / O_sum
print(O_sum[0,:])
print(O_softmax.shape)
print(O_softmax[0,:])

In [ ]:
print(O_softmax.shape)

In [ ]:
loss = O_softmax[ torch.arange(O_softmax.shape[0]), y_feats]
loss = -loss.log().mean()

In [ ]:
for i in range(100):
    O = x_feats @ W
    O = O.exp()
    O_sum = O.sum(1, keepdim=True)
    O_softmax = O / O_sum
    loss = O_softmax[ torch.arange(O_softmax.shape[0]), y_feats]
    loss = -loss.log().mean()
    loss.backward()
    loss.zero_grad()
    print(loss)



None
